# Rabbits and foxes

There are initially 400 rabbits and 200 foxes on a farm (but it could be two cell types in a 96 well plate or something, if you prefer bio-engineering analogies). Plot the concentration of foxes and rabbits as a function of time for a period of up to 600 days. The predator-prey relationships are given by the following set of coupled ordinary differential equations:

\begin{align}
\frac{dR}{dt} &= k_1 R - k_2 R F \tag{1}\\
\frac{dF}{dt} &= k_3 R F - k_4 F \tag{2}\\
\end{align}

* Constant for growth of rabbits $k_1 = 0.015$ day<sup>-1</sup>
* Constant for death of rabbits being eaten by foxes $k_2 = 0.00004$ day<sup>-1</sup> foxes<sup>-1</sup>
* Constant for growth of foxes after eating rabbits $k_3 = 0.0004$ day<sup>-1</sup> rabbits<sup>-1</sup>
* Constant for death of foxes $k_4 = 0.04$ day<sup>-1</sup>

*This problem is based on one from Chapter 1 of H. Scott Fogler's textbook "Essentials of Chemical Reaction Engineering".*


In [35]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

# Now let's try some Kinetic Monte Carlo

We wish to implement a Kinetic Monte Carlo algorithm to simulate the same situation. See https://en.wikipedia.org/wiki/Kinetic_Monte_Carlo for details.

We'll assume the numbers of rabbits and foxes (starting at 400 and 200) are actual rabbits and foxes, not "rabbit densities" for example, and so must always remain integers: you can't have half a rabbit or half a fox.

There are four events, with rates that are straightforward to calculate, so the rejection-free algorithm is suitable:
* `rabbit_birth = k1 * rabbits`
* `rabbit_death = k2 * rabbits * foxes`
* `fox_birth = k3 * rabbits * foxes`
* `fox_death = k4 * foxes`


Use a Kinetic Monte Carlo simulation(s) running for 600 days to determine
1. The expected location of the second peak in foxes (eg. 425 days, 2800 foxes), on occasions that there is one (eg. if there's a peak that's  >200 days and >100 foxes)
2. The interquartile range of the second peak in foxes (eg. 411-443 days, 2700-3120 foxes).
3. The probability that the foxes die out before 600 days are complete

Make sure you've done enough simulations to be suitably confident in your answers (given the precision you think appropriate).

# Your turn!

In [73]:
#from kmc-tips.py
import random
random.seed(8) # Fix seed so results don't change every time I execute
np.random.seed(8) # same thing for numpy

#define drdt and dfdt coefficients
k1 = 0.015
k2 = 0.00004
k3 = 0.0004
k4 = 0.04

def kmc():
    """
    performs kmc simulation
    """
    # 1. set t=0
    times = []
    time = 0
    times.append(0)
    end_time = 600

    # 2. initial state k
    rabbits = []
    foxes = []
    rabbits.append(400)
    foxes.append(200)
    
    i = 0 #counter for arrays
    
    # 10. Return to step 3
    while (time < 600):        
        # 3. list possible transitions rates from state k to generic state i 
        rabbit_birth = k1 * rabbits[i]               #positive term from dr/dt
        rabbit_death = k2 * rabbits[i] * foxes[i]    #negative term from dr/dt
        fox_birth = k3 * rabbits[i] * foxes[i]       #positive term from df/dt
        fox_death = k4 * foxes[i]                    #negative term from dr/dt
        #print(rabbit_birth)
        #print(rabbit_death)
        #print(fox_birth)
        #print(fox_death)

        # 4. cumulative function
        sum_rates = rabbit_birth + rabbit_death + fox_birth + fox_death
        RB_bound = rabbit_birth #bound for rabbit_birth
        RD_bound = rabbit_birth + rabbit_death #bound for rabbit_death
        FB_bound = rabbit_birth + rabbit_death + fox_birth #bound for fox_birth
        FD_bound = rabbit_birth + rabbit_death + fox_birth + fox_death #bound for fox_death

        # 5. get random uniform number u (0,1]
        choice = random.uniform(0, sum_rates)

        # 6. find event, i, to carry out
        # 7. carry out event i and update state k to state i
        if (choice < RB_bound):
            rabbits.append(rabbits[i]+1)
            foxes.append(foxes[i])
        elif ((choice >= RB_bound) & (choice < RD_bound)):
            rabbits.append(rabbits[i]-1)
            foxes.append(foxes[i])
        elif ((choice >= RD_bound) & (choice < FB_bound)):
            rabbits.append(rabbits[i])
            foxes.append(foxes[i]+1)
        else:
            rabbits.append(rabbits[i])
            foxes.append(foxes[i]-1)
                #I think this makes sense
        i = i+1 #increment i
        #print(i)

        # 8. get new random uniform number u' (0,1]
        # 9. update time with time increment function of u'
        wait_time = random.expovariate(sum_rates)
        time += wait_time
        times.append(time)
        
            #using this time increment method from kmc_tips.py
        
        
            
    times = np.array(times)
    rabbits = np.array(rabbits)
    foxes = np.array(foxes)
        
    
    return times, rabbits, foxes

def solve(n):
    """
    solves for:
        -average location of second maximum of fox population
        -interquartile range of the second maximum of fox population
        -probability of foxes dying out before day 600
    given desired number of iterations of the kmc simulation
    """
    
    sp_time = [] #array to hold day of second maximum in fox population per iteration
    sp_foxes = [] #array to hold second maximum in fox population per iteration
    f_extinct = 0 #counter for number of times foxes die out
    
    for i in range (1, n+1):   
        #run KMC simulation and import output arrays
        t, r, f = kmc()

        #check for fox extinction
        fx_counter = 1 #counter for limit increment of +1 on f_extinct counter per kmc simulation
        for j in range (1, len(f)):
            if ((f[j] == 0) & (f[j-1] != 0) & (fx_counter == 1)):
                f_extinct += 1
                fx_counter = 0
                
        #locate second maximum in output fox population array with criterion of >1000 foxes
        max_counter = 1 #counter to note that the first maximum is passed before reaching the second maximum
        index = 0 #to hold index of second maximum
        for k in range ( len(f)-1 ):
            dfdt = (f[k+1] - f[k]) #calculates the change between each record of fox population
            
            #to detect positive dfdt
            if (dfdt > 0):
                delta_f = 1 #to denote that fox population is increasing
             
            #when dfdt becomes negative we are concerned with three possibilities:
                #1: it is the second maximum 
                #2: it is the first maximum
                #3: it is not a maximum 
            #since 1 is the most selective, we check them in precisely this order
            #to detect second maximum >1000 foxes
            if ((dfdt < 0) & (delta_f == 1) & (f[k] > 1000) & (max_counter == 0)):
                delta_f = -1 #to denote that fox population is decreasing
                index = k #save location of this second maximum
                
            #to detect first maximum >1000 foxes
            if ((dfdt < 0) & (delta_f == 1) & (f[k] > 1000)):
                delta_f = -1 #to denote that fox population is decreasing
                max_counter = 0
                
            #in all other cases
            if (dfdt < 0):
                delta_f = -1 #to denote that fox population is decreasing
            
            #note that if fox population is constant, delta_f is unchanged; 
            #this way, we can track when dfdt changes sign
            #when dfdt changes from (+) to (-), this is a local maximum
            
        #save both time and value of the second maximum
        if (index != 0):
            sp_time.append(t[index])
            sp_foxes.append(f[index])
        #otherwise, no second maximum has been detected
    
    #calculate average probability of fox extinction
    prob_fox_extinct = (f_extinct/n)
    
    #calculate average time of and second maximum in fox population
    avg_sp_foxes = 0
    avg_sp_time = 0
    sp_foxes = np.array(sp_foxes)
    sp_time = np.array(sp_time)
    if (len(sp_foxes) > 0):
        avg_sp_foxes = sum(sp_foxes)/len(sp_foxes)
        avg_sp_time = sum(sp_time/len(sp_time))
    
    return prob_fox_extinct, avg_sp_foxes, avg_sp_time
    #unsure about interquartile range of second maximum
    
        
        
    

    



In [74]:
solve(100)

(0.74, 1013.56, 171.65890131908938)